# Run C# and Pwsh in the same Notebook

This notebook will run C# and pwsh in the saem notebook and pass values between them

We will see how long it takes between C# and pwsh to count files in a directory. First create 10,000 files

In [211]:
$playdir = "$Env:Temp\pwshplaydir"
if (-not (Test-Path $playdir)) {
    New-Item $playdir -ItemType Directory
    $10k = 0..10000
    $10k | ForEach-Object -ThrottleLimit 10 -Parallel {
        $PlayFileName = $Using:playdir + "\$PSItem.txt"
        $Null = New-Item $PlayFileName -ItemType File -Force
    }
}

Start a stopwatch

In [212]:
$Stopwatch = [diagnostics.stopwatch]::StartNew()

Get the file count with c# using the magic command `#!csharp`

In [213]:
#!csharp
using System.IO; 
using System;
int fileCount = Directory.GetFiles(@"C:\Users\mrrob\AppData\Local\Temp\pwshplaydir", "*", SearchOption.TopDirectoryOnly).Length;
Console.WriteLine($"Number of Files : {fileCount}\n");

Number of Files : 10001



In [214]:
$Stopwatch.Stop()
$csharpmillisecs = $Stopwatch.Elapsed.Milliseconds

Count the same files with pwsh

In [215]:
$Stopwatch = [diagnostics.stopwatch]::StartNew()
$FileCount = (Get-Childitem C:\Users\mrrob\AppData\Local\Temp\pwshplaydir).Count
Write-Output "The File Count is $FileCount"

The File Count is 10001


In [216]:
$Stopwatch.Stop()
$pwshmillisecs = $Stopwatch.Elapsed.Milliseconds

Which was quicker ?

In [217]:
function Which-WasQuicker {
    if($pwshmillisecs -lt $csharpmillisecs ){
        $percquicker = [math]::Round(($pwshmillisecs / $csharpmillisecs)  * 100, 2)
        $string = @"
        '<c="green">Pwsh in $pwshmillisecs milliseconds</c>was quicker than <c="red">c sharp in $csharpmillisecs milliseconds</c>in $percquicker % '
"@
    }else{
        $percquicker = [math]::Round(($csharpmillisecs / $pwshmillisecs )  * 100, 2)
        $string = @"
        '<c="green">c sharp in $csharpmillisecs milliseconds</c>was quicker than <c="red">Pwsh in $pwshmillisecs milliseconds</c>in $percquicker % '
"@      
    }
    Write-PSFHostColor -String $string  -DefaultColor DarkBlue
}

In [218]:
Which-WasQuicker

        '

c sharp in 133 milliseconds

was quicker than 

Pwsh in 343 milliseconds

in 38.78 % '


In [219]:
#!csharp
using System.IO; 
using System;
string value;
string File_Countfromenv;
int fileCount = Directory.GetFiles(@"C:\Users\mrrob\AppData\Local\Temp\pwshplaydir", "*", SearchOption.TopDirectoryOnly).Length;
value = Convert.ToString(fileCount);
Environment.SetEnvironmentVariable("File_Count", value);
File_Countfromenv = Environment.GetEnvironmentVariable("File_Count");
Console.WriteLine($"Number of Files from csharp :{File_Countfromenv}\n");

Number of Files from csharp :10001



In [220]:
Write-Output "The File Count is $Env:File_Count using variable from csharp in pwsh"

The File Count is 10001 using variable from csharp in pwsh


In [221]:
pwd

Path


----


C:\Users\mrrob
